#### 1. Import pandas library

In [131]:
import pandas as pd

#### 2. Import pymysql and sqlalchemy as you have learnt in the lesson of importing/exporting data 


In [132]:
import pymysql
from sqlalchemy import create_engine
import getpass

#### 3. Create a mysql engine to set the connection to the server. Check the connection details in [this link](https://relational.fit.cvut.cz/search?tableCount%5B%5D=0-10&tableCount%5B%5D=10-30&dataType%5B%5D=Numeric&databaseSize%5B%5D=KB&databaseSize%5B%5D=MB)

In [133]:
password= getpass.getpass("Insert your mysql root password: ")
engine = create_engine('mysql+pymysql://guest:{}@relational.fit.cvut.cz/stats'.format(password))
print("Connected to server!")


Insert your mysql root password: ········
Connected to server!


#### 4. Import the users table 

In [134]:
query= """SELECT * FROM users"""
users=pd.read_sql_query(query,engine)

#### 5. Rename Id column to userId

In [135]:
df.columns
users=users.rename(columns={'Id':'userId'})



#### 6. Import the posts table. 

In [142]:
query= """SELECT * FROM posts"""
posts=pd.read_sql_query(query,engine)

#### 7. Rename Id column to postId and OwnerUserId to userId

In [145]:
posts=posts.rename(columns={'Id':"postId","OwnerUserId":"userID"})
print(posts.columns)

Index(['postId', 'PostTypeId', 'AcceptedAnswerId', 'CreaionDate', 'Score',
       'ViewCount', 'Body', 'userID', 'LasActivityDate', 'Title', 'Tags',
       'AnswerCount', 'CommentCount', 'FavoriteCount', 'LastEditorUserId',
       'LastEditDate', 'CommunityOwnedDate', 'ParentId', 'ClosedDate',
       'OwnerDisplayName', 'LastEditorDisplayName'],
      dtype='object')


#### 8. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userID,ViewCount,CommentCount

In [146]:
users_colu=["userId","Reputation","Views","UpVotes","DownVotes"]

a=users[users_colu]

post_colum=["postId","Score","userID","ViewCount","CommentCount"]

b=posts[post_colum]

#### 8. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [164]:
union=a.merge (b, left_on="userId", right_on="userID")
union.head()

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,userID,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,-1.0,NaN,0
1,-1,1,0,5007,1920,8576,0,-1.0,NaN,0
2,-1,1,0,5007,1920,8578,0,-1.0,NaN,0
3,-1,1,0,5007,1920,8981,0,-1.0,NaN,0
4,-1,1,0,5007,1920,8982,0,-1.0,NaN,0


#### 9. How many missing values do you have in your merged dataframe? On which columns?

In [176]:
valores_nulos = union.isnull().sum()
print(valores_nulos)
#Los valores null se agrupan en la columna ViewCount


userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId              0
Score               0
userID              0
ViewCount       48396
CommentCount        0
dtype: int64


#### 10. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [185]:
#Datos null 48396
#Datos totales de ViewCount 90584
#Viendo la tabla se ve que la mayoria de valores NaN son de la columna de usuarios, 
#que no disponen de contador de visitas. Ya que no tendría tanto valor como 
#poder calcularlos en los post. Pero eliminar la columna entera, implicaria quitar 
#los de los post y esos si tienen valor.



display(union)

suma_vie=union["ViewCount"]
print(len(suma_vie))
print(union['ViewCount'].sum())

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,userID,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,-1.0,NaN,0
1,-1,1,0,5007,1920,8576,0,-1.0,NaN,0
2,-1,1,0,5007,1920,8578,0,-1.0,NaN,0
3,-1,1,0,5007,1920,8981,0,-1.0,NaN,0
4,-1,1,0,5007,1920,8982,0,-1.0,NaN,0
5,-1,1,0,5007,1920,9857,0,-1.0,NaN,0
6,-1,1,0,5007,1920,9858,0,-1.0,NaN,0
7,-1,1,0,5007,1920,9860,0,-1.0,NaN,0
8,-1,1,0,5007,1920,10130,0,-1.0,NaN,0
9,-1,1,0,5007,1920,10131,0,-1.0,NaN,0


90584
23484210.0


#### 11. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [186]:
union['ViewCount'] = union['ViewCount'].astype(object)
union['ViewCount'].dtype

23484210.0


#### Bonus: Identify extreme values in your merged dataframe as you have learned in class, create a dataframe called outliers with the same columns as our data set and calculate the bounds. The values of the outliers dataframe will be the values of the merged_df that fall outside that bounds. You will need to save your outliers dataframe to a csv file on your-code folder.